In [1]:
from flask import Flask,render_template,request
import numpy as np

In [2]:
from modules.model import FCNN, FCNN_controller
from modules.env import Gomoku, Observer
from modules.agent import FNAgent

In [3]:
class Refree:
    
    def __init__(self):
        self.env = None
        self.agent = None
        self.player_ids = None
        self.is_player_turn = False
        self.state = None
        self.options = None
        self.done = False
        self.result = "対戦中"
    
    @classmethod
    def load(cls, env, agent):
        refree = cls()
        refree.env = env
        refree.agent = agent
        return refree
    
    def reset(self):
        self.done = False
        self.result = "対戦中"
        self.state, self.options = self.env.reset()
        if np.random.random() < 0.5:
            self.player_ids = {"player" : 1, "agent" : -1}
            self.is_player_turn = True
        else:
            self.player_ids = {"player" : -1, "agent" : 1}
            self.is_player_turn = False
            self.agent_turn()
            
    def agent_turn(self):
        if self.done or self.is_player_turn:
            self.state, reward, self.done, self.options = self.env.step(self.player_ids["agent"], None)
            return
        action = self.agent.policy(self.state, self.options)
        self.state, reward, self.done, self.options = self.env.step(self.player_ids["agent"], action)
        if len(self.options) != 0:
            self.is_player_turn = True
        if reward == 1:
            self.result = "負け"
        elif reward == -0.3:
            self.result = "引き分け"
        
    def player_turn(self, action):
        if self.done or not self.is_player_turn:
            self.state, reward, self.done, self.options = self.env.step(self.player_ids["player"], None)
            return
        self.state, reward, self.done, self.options = self.env.step(self.player_ids["player"], action)
        if len(self.options) != 0:
            self.is_player_turn = False
        if reward == 1:
            self.result = "勝ち"
        elif reward == -0.3:
            self.result = "引き分け"

In [4]:
def init_gomoku_refree():
    env = Observer.load(Gomoku(3))
    fcnn = FCNN(env.dim_state+2, 1)
    model = FCNN_controller(fcnn)
    model.load_weight("data/gomoku/dnn/10_model_fcnn")
    agent = DNQAgent.load(model, None, 0)
    
    refree = Refree.load(env, agent)
    refree.reset()
    
    return refree

In [ ]:
def init_othello_refree():
    env = OthelloObserver.load(Othello())
    agent = DNQAgent(0)
    
    refree = Refree.load(env, agent)
    refree.reset()
    
    return refree

In [5]:
gomoku_refree = init_gomoku_refree()
othello_refree = init_othello_refree()

In [6]:
app = Flask(__name__)

@app.route("/", methods=["GET"])
def root_get():
    gomoku_refree.reset()
    return render_template("index.html", state=gomoku_refree.env.state().reshape(3, 3).tolist(), options=gomoku_refree.options.tolist(), 
    order=gomoku_refree.player_ids["player"], result=gomoku_refree.result)

@app.route("/", methods=["POST"])
def root_post():
    action = int(request.form.get("action"))
    gomoku_refree.player_turn(action)
    gomoku_refree.agent_turn()
    return render_template("index.html", state=gomoku_refree.env.state().reshape(3, 3).tolist(), options=gomoku_refree.options.tolist(), 
    order=gomoku_refree.player_ids["player"], result=gomoku_refree.result)
    

@app.route("/othello", methods=["GET"])
def othello_get():
    othello_refree.reset()
    return render_template("othello.html", state=othello_refree.env.state().reshape(8, 8).tolist(), options=othello_refree.options.tolist(), 
    order=othello_refree.player_ids["player"], result=othello_refree.result)

@app.route("/othello", methods=["POST"])
def othello_post():
    action = int(request.form.get("action"))
    othello_refree.player_turn(action)
    while not othello_refree.is_player_turn:
        othello_refree.agent_turn()
    return render_template("othello.html", state=othello_refree.env.state().reshape(8, 8).tolist(), options=othello_refree.options.tolist(), 
    order=othello_refree.player_ids["player"], result=othello_refree.result)

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Dec/2021 17:44:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 17:44:39] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [31/Dec/2021 17:44:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 17:44:40] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [31/Dec/2021 17:44:42] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 17:44:42] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [31/Dec/2021 17:44:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 17:44:43] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [31/Dec/2021 17:44:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 17:44:48] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [31/Dec/2021 17:44:50] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 17:44:50] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [31/Dec/2021 17:44:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 17:44:51